# Woodcock (Scolopax Minor) Migration:

### This notebook is used to demonstrate the use of imageio to make .mp4 and .gif files where the migration can be visualized.  This is similar to using matplotlib's animation function, but works better with large amounts of data / complex plots.

### I've always thought migrations were cool; I hope you enjoy the data visualization!

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from datetime import datetime, timedelta

## Read in the original and processed datasets.

In [2]:
# Load the data
df = pd.read_csv("USGS Woodcock Migration.csv")

In [3]:
# Drop birds with low observation count
df = df.groupby("individual-local-identifier").filter(lambda x: len(x) > 100)
# Get list of bird names
birds = df['individual-local-identifier'].unique()

In [4]:
dfinterp = pd.read_csv('WoodcockData.csv')

## Create the frames for the movie.

### This takes some time as it needs to make a lot of frames.  In each frame, I included a fading tail to make it easier to follow the bird's movement.  The fading tails add to the time to run this cell as well.

In [5]:
TailLength = 168.
Step = 36
k = 0

while k < (len(dfinterp) - TailLength)/Step:
    plt.figure(figsize=(12,9))
    m = Basemap(llcrnrlon=-100., llcrnrlat=25., urcrnrlon=-60., urcrnrlat=57.,
               projection='lcc', lat_1=20., lat_2=40., lon_0=-60.,
               resolution='l', area_thresh=1000.)
    
    m.drawcoastlines()
    m.drawcountries()
    m.drawstates()
    m.drawparallels(np.arange(10,70,10), labels=[1,1,0,0])
    m.drawmeridians(np.arange(-100,0,10), labels=[0,0,0,1])
    
    for i in birds:
        j=0
        while j <= TailLength:
            col = str(1 - j/TailLength)
            m.scatter(dfinterp.loc[int(j+k*Step), i+'_lon'], dfinterp.loc[int(j+k*Step), i+'_lat'], 
                      latlon=True, marker='o', color=col)
            j += 1
            
    plt.title(dfinterp.loc[int(j+k*Step), 'timestamp'][5:7]+'/'+dfinterp.loc[int(j+k*Step), 'timestamp'][0:4], 
              fontdict = {'fontsize' : 20})
    plt.savefig('Migration'+str(k)+'.png')
    
    plt.cla()
    plt.clf()
    plt.close('all')
    del m
    
    k += 1

## Import imageio and create the movie.

### The line of code near the end of the cell can be uncommented to make a .gif file instead.  Note that the .mp4 file is significantly smaller in size.

In [6]:
import imageio

images = []

for i in range(0,931):
    name = 'Migration' + str(i) + '.png'
    images.append(imageio.imread(name))
    
# imageio.mimsave('MigrationMovie.gif', images, duration=0.1)
imageio.mimsave('MigrationMovie.mp4', images)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (864, 648) to (864, 656) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
